# SGAN - Semi-Supervised Learning with Generative Adversarial Networks


Ref.:
ODENA, Augustus.  
Semi-supervised learning with generative adversarial networks.  
arXiv preprint arXiv:1606.01583, 2016.

![SGAN](sgan.png)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


## Generator

In [3]:
def build_generator(latent_dim):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

# Discriminator

In [4]:
def build_discriminator(img_shape, num_classes):

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.summary()

    img = Input(shape=img_shape)

    features = model(img)
    valid = Dense(1, activation="sigmoid")(features)
    label = Dense(num_classes + 1, activation="softmax")(features)

    return Model(img, [valid, label])

## helper

In [5]:
def sample_images(G, latent_dim, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [6]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

In [7]:
def save_model(G, D, combined):
    save(G, "mnist_sgan_generator")
    save(D, "mnist_sgan_discriminator")
    save(combined, "mnist_sgan_adversarial")

## Train

In [8]:
def train(G, D, combined,
          num_classes, latent_dim,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, y_train), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    y_train = y_train.reshape(-1, 1)

    # Class weights:
    # To balance the difference in occurences of digit class labels.
    # 50% of labels that the discriminator trains on are 'fake'.
    # Weight = 1 / frequency
    half_batch = batch_size // 2
    cw1 = {0: 1, 1: 1}
    cw2 = {i: num_classes / half_batch for i in range(num_classes)}
    cw2[num_classes] = 1 / half_batch

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = G.predict(noise)

        # One-hot encoding of labels
        labels = to_categorical(y_train[idx], num_classes=num_classes + 1)
        fake_labels = to_categorical(np.full((batch_size, 1), num_classes), num_classes=num_classes + 1)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
        d_loss_fake = D.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, latent_dim, epoch)


## main()

In [9]:
for d in ['images']:
    if not os.path.exists(d):
        os.makedirs(d)

In [10]:
# initialization

In [11]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100

In [12]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
# Build and compile the discriminator
D = build_discriminator(img_shape, num_classes)
D.compile(loss=['binary_crossentropy', 'categorical_crossentropy'],
          loss_weights=[0.5, 0.5],
          optimizer=optimizer,
          metrics=['accuracy']
          )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)         

In [14]:
# Build the generator
G = build_generator(latent_dim)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)      

In [15]:
# The generator takes noise as input and generates imgs
noise = Input(shape=(100,))
img = G(noise)

In [16]:
# For the combined model we will only train the generator
D.trainable = False

In [17]:
# The valid takes generated images as input and determines validity
valid, _ = D(img)

In [18]:
# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model(noise, valid)
combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

In [19]:
# epochs=20000
epochs=5000
train(G, D, combined,
      num_classes, latent_dim,
      epochs=epochs,
      batch_size=32, sample_interval=50)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.852611, acc: 43.75%, op_acc: 25.00%] [G loss: 0.790520]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.836038, acc: 48.44%, op_acc: 10.94%] [G loss: 0.734077]
2 [D loss: 0.793688, acc: 53.12%, op_acc: 15.62%] [G loss: 0.904134]
3 [D loss: 0.760562, acc: 65.62%, op_acc: 12.50%] [G loss: 1.056291]
4 [D loss: 0.671614, acc: 79.69%, op_acc: 14.06%] [G loss: 1.008282]
5 [D loss: 0.624139, acc: 84.38%, op_acc: 12.50%] [G loss: 1.031089]
6 [D loss: 0.540043, acc: 92.19%, op_acc: 21.88%] [G loss: 1.009021]
7 [D loss: 0.619452, acc: 81.25%, op_acc: 17.19%] [G loss: 0.746406]
8 [D loss: 0.554254, acc: 96.88%, op_acc: 15.62%] [G loss: 0.788827]
9 [D loss: 0.525718, acc: 92.19%, op_acc: 20.31%] [G loss: 0.536111]
10 [D loss: 0.526477, acc: 93.75%, op_acc: 23.44%] [G loss: 0.494904]
11 [D loss: 0.489427, acc: 96.88%, op_acc: 26.56%] [G loss: 0.245351]
12 [D loss: 0.434061, acc: 96.88%, op_acc: 40.62%] [G loss: 0.309138]
13 [D loss: 0.488002, acc: 92.19%, op_acc: 34.38%] [G loss: 0.202831]
14 [D loss: 0.417702, acc: 95.31%, op_acc: 54.69%] [G loss: 0.163831]
15 [D loss: 0.399317, acc: 96

118 [D loss: 0.630491, acc: 53.12%, op_acc: 45.31%] [G loss: 1.930408]
119 [D loss: 0.691692, acc: 42.19%, op_acc: 46.88%] [G loss: 1.731977]
120 [D loss: 0.513790, acc: 57.81%, op_acc: 51.56%] [G loss: 1.951701]
121 [D loss: 0.725316, acc: 39.06%, op_acc: 45.31%] [G loss: 1.657020]
122 [D loss: 0.465930, acc: 62.50%, op_acc: 46.88%] [G loss: 1.678019]
123 [D loss: 0.663426, acc: 34.38%, op_acc: 42.19%] [G loss: 1.414584]
124 [D loss: 0.544723, acc: 67.19%, op_acc: 43.75%] [G loss: 1.708964]
125 [D loss: 0.617288, acc: 50.00%, op_acc: 40.62%] [G loss: 1.652477]
126 [D loss: 0.535955, acc: 56.25%, op_acc: 53.12%] [G loss: 1.350711]
127 [D loss: 0.522719, acc: 57.81%, op_acc: 48.44%] [G loss: 1.676039]
128 [D loss: 0.590074, acc: 57.81%, op_acc: 43.75%] [G loss: 1.506164]
129 [D loss: 0.584025, acc: 45.31%, op_acc: 45.31%] [G loss: 1.964328]
130 [D loss: 0.485878, acc: 67.19%, op_acc: 56.25%] [G loss: 1.395994]
131 [D loss: 0.535027, acc: 60.94%, op_acc: 43.75%] [G loss: 1.422047]
132 [D

...

4465 [D loss: 0.344356, acc: 67.19%, op_acc: 64.06%] [G loss: 1.253406]
4466 [D loss: 0.407830, acc: 65.62%, op_acc: 57.81%] [G loss: 1.255215]
4467 [D loss: 0.343163, acc: 73.44%, op_acc: 59.38%] [G loss: 1.225855]
4468 [D loss: 0.416169, acc: 51.56%, op_acc: 56.25%] [G loss: 1.070942]
4469 [D loss: 0.409919, acc: 64.06%, op_acc: 60.94%] [G loss: 0.989918]
4470 [D loss: 0.318168, acc: 70.31%, op_acc: 64.06%] [G loss: 1.182139]
4471 [D loss: 0.360411, acc: 62.50%, op_acc: 57.81%] [G loss: 1.078342]
4472 [D loss: 0.417974, acc: 57.81%, op_acc: 56.25%] [G loss: 1.267419]
4473 [D loss: 0.381832, acc: 60.94%, op_acc: 59.38%] [G loss: 1.194542]
4474 [D loss: 0.354939, acc: 71.88%, op_acc: 57.81%] [G loss: 1.160678]
4475 [D loss: 0.385739, acc: 68.75%, op_acc: 59.38%] [G loss: 1.180662]
4476 [D loss: 0.352297, acc: 62.50%, op_acc: 57.81%] [G loss: 1.190380]
4477 [D loss: 0.403164, acc: 65.62%, op_acc: 54.69%] [G loss: 0.967645]
4478 [D loss: 0.401509, acc: 64.06%, op_acc: 53.12%] [G loss: 1.

4579 [D loss: 0.429644, acc: 54.69%, op_acc: 62.50%] [G loss: 1.285847]
4580 [D loss: 0.328816, acc: 71.88%, op_acc: 59.38%] [G loss: 1.423830]
4581 [D loss: 0.346399, acc: 64.06%, op_acc: 62.50%] [G loss: 1.131815]
4582 [D loss: 0.401498, acc: 67.19%, op_acc: 57.81%] [G loss: 1.064947]
4583 [D loss: 0.324699, acc: 76.56%, op_acc: 68.75%] [G loss: 1.105434]
4584 [D loss: 0.344593, acc: 71.88%, op_acc: 56.25%] [G loss: 1.564781]
4585 [D loss: 0.346775, acc: 67.19%, op_acc: 59.38%] [G loss: 1.104408]
4586 [D loss: 0.316625, acc: 73.44%, op_acc: 60.94%] [G loss: 1.278524]
4587 [D loss: 0.349556, acc: 59.38%, op_acc: 65.62%] [G loss: 0.897153]
4588 [D loss: 0.395406, acc: 65.62%, op_acc: 59.38%] [G loss: 1.204335]
4589 [D loss: 0.397996, acc: 59.38%, op_acc: 60.94%] [G loss: 1.096410]
4590 [D loss: 0.327640, acc: 73.44%, op_acc: 64.06%] [G loss: 1.336903]
4591 [D loss: 0.335309, acc: 73.44%, op_acc: 62.50%] [G loss: 1.173683]
4592 [D loss: 0.349016, acc: 64.06%, op_acc: 64.06%] [G loss: 1.

4693 [D loss: 0.294714, acc: 75.00%, op_acc: 60.94%] [G loss: 1.240288]
4694 [D loss: 0.362595, acc: 64.06%, op_acc: 64.06%] [G loss: 1.039145]
4695 [D loss: 0.339247, acc: 68.75%, op_acc: 67.19%] [G loss: 1.223227]
4696 [D loss: 0.410019, acc: 56.25%, op_acc: 60.94%] [G loss: 1.092298]
4697 [D loss: 0.308314, acc: 81.25%, op_acc: 62.50%] [G loss: 1.322597]
4698 [D loss: 0.389871, acc: 59.38%, op_acc: 68.75%] [G loss: 1.099890]
4699 [D loss: 0.365046, acc: 60.94%, op_acc: 57.81%] [G loss: 0.972697]
4700 [D loss: 0.380732, acc: 64.06%, op_acc: 51.56%] [G loss: 1.086460]
4701 [D loss: 0.362383, acc: 71.88%, op_acc: 56.25%] [G loss: 1.093761]
4702 [D loss: 0.349857, acc: 67.19%, op_acc: 67.19%] [G loss: 1.181374]
4703 [D loss: 0.384295, acc: 65.62%, op_acc: 68.75%] [G loss: 1.252984]
4704 [D loss: 0.306939, acc: 76.56%, op_acc: 62.50%] [G loss: 1.087533]
4705 [D loss: 0.333715, acc: 76.56%, op_acc: 56.25%] [G loss: 1.358233]
4706 [D loss: 0.415239, acc: 65.62%, op_acc: 46.88%] [G loss: 0.

4807 [D loss: 0.439599, acc: 48.44%, op_acc: 57.81%] [G loss: 1.195919]
4808 [D loss: 0.405098, acc: 62.50%, op_acc: 65.62%] [G loss: 1.161799]
4809 [D loss: 0.394686, acc: 62.50%, op_acc: 59.38%] [G loss: 1.093107]
4810 [D loss: 0.372417, acc: 57.81%, op_acc: 65.62%] [G loss: 1.325995]
4811 [D loss: 0.421636, acc: 59.38%, op_acc: 62.50%] [G loss: 1.073761]
4812 [D loss: 0.390455, acc: 62.50%, op_acc: 60.94%] [G loss: 1.226436]
4813 [D loss: 0.312995, acc: 78.12%, op_acc: 70.31%] [G loss: 1.533618]
4814 [D loss: 0.419436, acc: 48.44%, op_acc: 67.19%] [G loss: 1.012317]
4815 [D loss: 0.414004, acc: 54.69%, op_acc: 57.81%] [G loss: 1.147648]
4816 [D loss: 0.397064, acc: 54.69%, op_acc: 57.81%] [G loss: 1.041413]
4817 [D loss: 0.467246, acc: 51.56%, op_acc: 54.69%] [G loss: 1.302058]
4818 [D loss: 0.332813, acc: 76.56%, op_acc: 59.38%] [G loss: 0.940879]
4819 [D loss: 0.366075, acc: 71.88%, op_acc: 54.69%] [G loss: 1.168519]
4820 [D loss: 0.502673, acc: 51.56%, op_acc: 46.88%] [G loss: 0.

4921 [D loss: 0.351819, acc: 68.75%, op_acc: 56.25%] [G loss: 1.292328]
4922 [D loss: 0.387024, acc: 59.38%, op_acc: 57.81%] [G loss: 1.179340]
4923 [D loss: 0.370130, acc: 64.06%, op_acc: 57.81%] [G loss: 1.164813]
4924 [D loss: 0.328762, acc: 67.19%, op_acc: 67.19%] [G loss: 1.252285]
4925 [D loss: 0.364235, acc: 73.44%, op_acc: 62.50%] [G loss: 1.179555]
4926 [D loss: 0.457103, acc: 64.06%, op_acc: 51.56%] [G loss: 1.375780]
4927 [D loss: 0.322062, acc: 70.31%, op_acc: 67.19%] [G loss: 1.047063]
4928 [D loss: 0.328259, acc: 71.88%, op_acc: 60.94%] [G loss: 1.008881]
4929 [D loss: 0.281568, acc: 79.69%, op_acc: 76.56%] [G loss: 1.368791]
4930 [D loss: 0.386975, acc: 57.81%, op_acc: 54.69%] [G loss: 1.209825]
4931 [D loss: 0.326207, acc: 73.44%, op_acc: 64.06%] [G loss: 1.147925]
4932 [D loss: 0.324793, acc: 73.44%, op_acc: 64.06%] [G loss: 1.375264]
4933 [D loss: 0.357017, acc: 67.19%, op_acc: 62.50%] [G loss: 1.100159]
4934 [D loss: 0.293566, acc: 75.00%, op_acc: 70.31%] [G loss: 1.